# FVM WFTM-MPX LP

In [2]:
import os
import json
import pandas as pd

mpx_erc20_address = "0x66eEd5FF1701E6ed8470DC391F05e27B1d0657eb"
fvm_wftm_mpx_address = "0xF8eed2665FD11a8431fc41b2582fD5E72a1606f0"
fvm_wftm_mpx_gauge = "0xF89f367E0225fE68c7c28Fad0BaDc7f569987cFe"

with open("../data/fantom-addresses.json", "r") as f:
    addresses = json.load(f)
len(addresses)

4511

## FVM WFTM-MPX LP

First we obtain the MPX balance for the Fvm LP.
These balances are queried from the MPX ERC-20 contract.

**Note: the balance is 0 at the time of the snapshot, which predates the contract deployment.**

In [7]:
mpx_erc20_balances = pd.read_csv("../products/fantom-mpx-balances.csv")
try:
    pool_mpx_total = mpx_erc20_balances.query(f'address == "{fvm_wftm_mpx_address}"')
    pool_mpx_total = int(list(pool_mpx_total['balance'])[0])
except:
    pool_mpx_total = 0

pool_mpx_total / 1e18

0.0

Then we obtain the balances of the Fvm WFTM-MPX LP.
This represents the distribution of MPX provided for liquidity.

In [8]:
with open("../data/fantom-fvm-wftm-mpx.json", "r") as f:
    fvm_call_results = json.load(f)
len(fvm_call_results)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
fvm_lp_balances = {}
for key, value in fvm_call_results.items():
    if value > 0:
        fvm_lp_balances[parse_address(key)] = value

pool_lp_total = sum(fvm_lp_balances.values())
pool_lp_total / 1e18

851396.9011039473

## Fvm WFTM-MPX Gauges

Most of the LP is controlled by a gauge.
So, we must examine gauge contract balances to find the actual addresses that control the MPX.

In [10]:
with open("../data/fantom-fvm-gauge.json", "r") as f:
    fvm_gauge_call_results = json.load(f)
len(fvm_gauge_call_results)

4510

In [25]:
fvm_gauge_balances = {}
for key, value in fvm_gauge_call_results.items():
    if value > 0:
        fvm_gauge_balances[parse_address(key)] = value

gauge_lp_total = sum(fvm_gauge_balances.values())
sum(fvm_gauge_balances.values()) / 1e18

524974.0046707705

## Attribute MPX to LP and Gauge

Finally, we attribute MPX to original addresses.
The MPX controlled by each address is held in proportion to the LP tokens controlled by that address.

Step 1. Examine the LP and directly assign LP to non-gauge addresses.

In [36]:
fvm_lp_attributed_balances = {}
for key, value in fvm_call_results.items():
    if value > 0 and parse_address(key) != fvm_wftm_mpx_gauge:
        fvm_lp_attributed_balances[parse_address(key)] = value
sum(fvm_lp_attributed_balances.values()) / 1e18

1123.1288125757815

Step 2. Examine Gauges and assign LP to addresses.
First determine how much LP is controlled by a gauge, then assign LP based on proportion of gauge.

In [38]:
lp_controlled_by_gauge = fvm_lp_balances[fvm_wftm_mpx_gauge]
lp_controlled_by_gauge / 1e18

839123.0124607284

In [39]:
for key, value in fvm_gauge_call_results.items():
    if value > 0 and parse_address(key) != fvm_wftm_mpx_gauge:
        fvm_lp_attributed_balances[parse_address(key)] = (value / gauge_lp_total) * lp_controlled_by_gauge
sum(fvm_lp_attributed_balances.values()) / 1e18

840188.8691212935

In [42]:
(1 - ((pool_lp_total - sum(fvm_lp_balances.values())) / pool_lp_total)) * 100

100.0

We have attributed all of the LP.

## Final export of balances

In [46]:
df = pd.DataFrame.from_dict(fvm_lp_attributed_balances, orient="index", columns=["lp_balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "lp_balance"]]
df['mpx_balance'] = (df['lp_balance'] / pool_lp_total) * pool_mpx_total
df['lp_balance'] = df['lp_balance'].apply(lambda x: int(x)).values.tolist()
df['mpx_balance'] = df['mpx_balance'].apply(lambda x: int(x)).values.tolist()
df.to_csv("../products/fantom-fvm-balances.csv", index=False, float_format='%g')
sum(df['lp_balance']) / 1e18

848706.3446693354